# Evaluation criteria

The goal of this assignment is to get a view on your hands-on "data engineering" skills.  
At our company, our data scientists and engineers collaborate on projects.  
Your main focus will be creating performant & robust data flows.  
For a take-home-assignment, we cannot grant you access to our infrastructure.  
The assignement below measures your proficiency in general programming, data science & engineering tasks using python.  
Completion should not take more than half a day.

**We expect you to be proficient in:**
 * SQL queries (Sybase IQ system)
 * ETL flows (In collaboration with existing teams)
 * General python to glue it all together
 * Python data science ecosystem (Pandas + SKlearn)
 
**In this exercise we expect you to demonstrate your ability to / knowledge of:**
 * Building a data science runtime
 * PEP8 / Google python styleguide
 * Efficiently getting the job done
 * Choose meaningfull names for variables & functions
 * Writing maintainable code (yes, you might need to document some steps)
 * Help a data scientist present interactive results.
 * Offer predictions via REST api

# Setting-up a data science workspace

We allow you full freedom in setting up a data science runtime.  
The main objective is having a runtime where you can run this notebook and the code you will develop.  
You can choose for a local setup on your pc, or even a cloud setup if you're up for it.   

**In your environment, you will need things for:**
 * https request
 * python3 (not python2 !!)
 * (geo)pandas
 * interactive maps (e.g. folium, altair, ...)
 * REST apis
 
**Deliverables we expect**:
 * notebook with the completed assignment
 * list of packages for your runtime (e.g. yml or txt file)
 * evidence of a working API endpoint

# Importing packages

We would like you to put all your import statements here, together in 1 place.  
Before submitting, please make sure you remove any unused imports :-)  

In [ ]:
import pandas as pd
import numpy as np
import urllib.request, json
import unittest

# Data ingestion exercises

## Getting store location data from an API

**Goal:** Obtain a pandas dataframe  
**Hint:** You will need to normalise/flatten the json, because it contains multiple levels  
**API call:** https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places  

In [ ]:
def get_clp_places(url):
    with urllib.request.urlopen(url) as req:
        data = json.loads(req.read().decode())
    df = pd.json_normalize(data)
    return df

df_clp = get_clp_places("https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places")
df_clp.head(10)

### Quality checks

We would like you to add several checks on this data based on these constraints:  
 * records > 200
 * latitude between 49 and 52
 * longitude between 2 and 7
 
We dont want you to create a full blown test suite here, we're just gonna use 'asserts' from unittest

In [ ]:
tc = unittest.TestCase('__init__')
# records > 200
tc.assertGreater(len(df_clp.index), 200, "There should be more than 200 records")
# latitude between 49 and 52
tc.assertTrue(
    ((df_clp["geoCoordinates.latitude"] >= 49) & (df_clp["geoCoordinates.latitude"] <= 52)).all(), 
    "All stores should have a latitude between 49 and 52"
)
# longitude between 2 and 7
tc.assertTrue(
    ((df_clp["geoCoordinates.longitude"] >= 2) & (df_clp["geoCoordinates.longitude"] <= 7)).all(), 
    "All stores should have a longitude between 2 and 7"
)

### Feature creation

Create a new column "antwerpen" which is 1 for all stores in Antwerpen (province) and 0 for all others 

In [ ]:
df_clp["antwerpen"] = np.where((df_clp["address.postalcode"].astype(int) >= 2000) & (df_clp["address.postalcode"].astype(int) < 3000), 1, 0)
df_clp["antwerpen"].value_counts()

## Predict used car value

A datascientist in our team made a basic model to predict car prices.  
The model was saved to disk ('lgbr_cars.model') using joblib's dump fuctionality.  
Documentation states the model is a LightGBM Regressor, trained using the sk-learn api.  

**As engineer, your task it to expose this model as REST-api.** 

First, retrieve the model via the function below.  
Change the path according to your setup.  

In [ ]:
## your code goes here

def retrieve_model(path):
    raise NotImplementedError("You need to retrieve the trained model!")
    return trained_model

lgbr_cars = retrieve_model("../../models/lgbr_cars.model")

tc.assertEqual(str(type(lgbr_cars)),"<class 'lightgbm.sklearn.LGBMRegressor'>", type(lgbr_cars))

Now you have your trained model, lets do a functional test based on the parameters below.  
You have to present the parameters in this order.  

* vehicleType: coupe
* gearbox: manuell
* powerPS: 190
* model: NaN
* kilometer: 125000
* monthOfRegistration: 5 
* fuelType: diesel
* brand: audi

Based on these parameters, you should get a predicted value of 14026.35068804
However, the model doesnt accept string inputs, see the integer encoding below:

In [ ]:
model_test_input = [[3,1,190,-1,125000,5,3,1]]

In [ ]:
## your code goes here

def make_prediction(trained_model, single_input):
    raise NotImplementedError("You need to predict the value!")
    return predicted_value

predicted_value = make_prediction(lgbr_cars, model_test_input)

tc.assertAlmostEqual(predicted_value, 14026.35, places=2)

Now you got this model up and running, we want you to **expose it as a rest api.**  
We don't expect you to set up any authentication.  
We're not looking for beautiful inputs, just make it work.  
**Building this endpoint should NOT be done in a notebook, but in proper .py file(s)**

Once its up and running, use it to predict the following input:
* [-1,1,0,118,150000,0,1,38] ==> prediction should be 13920.70

## Geospatial data exercise
The goal of this exercise is to read in some data from a shape file and visualize it on a map
- The map should be dynamic. I want to zoom in and out to see more interesting aspects of the map
- We want you to visualize the statistical sectors within a distance of 2KM of your home location.

Specific steps to take:
- Read in the shape file
- Transform to WGS coordinates
- Create a distance function (Haversine)
- Create variables for home_lat, home_lon and perimeter_distance
- Calculate centroid for each nis district
- Calculate the distance to home for each nis district centroid 
- Figure out which nis districts are near your home
- Create dynamic zoomable map
- Visualize the nis districts near you (centroid <2km away), on the map


In [ ]:
# Some imports to help you along the way
import geopandas as gpd
import folium # you can use any viz library you prefer


In [ ]:
# part 1: Reading in the data
# get this file from https://statbel.fgov.be/sites/default/files/files/opendata/Statistische%20sectoren/sh_statbel_statistical_sectors_20200101.shp.zip 
df = gpd.read_file('../../data/external/shapefiles/AD_0_StatisticSector.shp')
df = df.to_crs({'init': 'epsg:4326'}) # change projection to wgs84 

# One of the data scientists discovered stackoverflow ;-) and copypasted something from https://gis.stackexchange.com/questions/166820/geopandas-return-lat-and-long-of-a-centroid-point
# A data science engineer should be able to speed this next code up

for i in range(0, len(df)):
    df.loc[i,'centroid_lon'] = df.geometry.centroid.x.iloc[i]
    df.loc[i,'centroid_lat'] = df.geometry.centroid.y.iloc[i]

In [ ]:
# Let's create some variables to indicate the location of your interest 
home_lat = NotImplemented
home_lon = NotImplemented
perimeter_distance = 2 # km

In [ ]:
# At some point we will need a distance function (google the Haversine formula, and implement it)

def haversine(lat1, lon1, lat2, lon2):
    raise NotImplementedError('Implement haversine')


Next, implement some sanity checks for your distance function 

In [ ]:
# implement sanity checks here

Now, create a dynamical map 

In [ ]:
# implementation of the map goes here